In [7]:
import os,sys,json,glob
import pandas as pd
import numpy as np
from xmljson import badgerfish as bf
from xml.etree.ElementTree import fromstring
from json import dumps

# đọc dữ liệu từ manaifest
data=bf.data(fromstring(open("dataset/manifest.xml","r").read()))
data=data['container']
data=data['testcase'] #Bao gồm tổng là 64,000 testcase, những trong 1 test lại có nhiều file
data[0]
ds=[]

In [8]:
for i in data:
  i=i['file'] 
  if isinstance(i,(list,tuple)):
    ds.extend(i)
  else: ds.append(i)
ds_pd=[]

In [9]:
for i in ds:
  path = i['@path'] 
  flaw =i.get("flaw",{"@line":None,"@name":None})
  cwe = path.split("__")[0]
  
  # nhưng file tồn tại nhiều CWE
  if isinstance(flaw, (list, tuple)):
    for fl in flaw:
      line = fl['@line']
      error_type = fl['@name']
      ds_pd.append({
          "File":path,
          "Line":line,
          "CWE":cwe,
      })
  else:
    line = flaw['@line']
    error_type = flaw['@name']
    ds_pd.append({
        "File":path,
        "Line":line,
        "CWE":cwe,
    })

In [10]:
# convert theo dạng Dataframa
# trên thực tế những file có flaw sẽ ít hơn số này
data_final=pd.DataFrame(ds_pd) 
data_final.CWE.unique().shape

(118,)

In [12]:
# Số testcase được định nghĩa là có lỗ hỏng
data_final=data_final[~data_final.Line.isna()]
data_final

,File,Line,CWE
0,CWE114_Process_Control__w32_char_connect_socke...,121.0,CWE114_Process_Control
1,CWE114_Process_Control__w32_char_connect_socke...,124.0,CWE114_Process_Control
2,CWE114_Process_Control__w32_char_connect_socke...,124.0,CWE114_Process_Control
3,CWE114_Process_Control__w32_char_connect_socke...,131.0,CWE114_Process_Control
4,CWE114_Process_Control__w32_char_connect_socke...,131.0,CWE114_Process_Control
...,...,...,...
106355,CWE90_LDAP_Injection__w32_wchar_t_listen_socke...,56.0,CWE90_LDAP_Injection
106357,CWE90_LDAP_Injection__w32_wchar_t_listen_socke...,51.0,CWE90_LDAP_Injection
106361,CWE90_LDAP_Injection__w32_wchar_t_listen_socke...,51.0,CWE90_LDAP_Injection
106365,CWE90_LDAP_Injection__w32_wchar_t_listen_socke...,154.0,CWE90_LDAP_Injection


In [13]:
# đếm kết quả theo từng CWE
sum = 0
for cwe in data_final['CWE'].unique():
  print(cwe, len(data_final[data_final['CWE'] == cwe]))
  sum += len(data_final[data_final['CWE'] == cwe])
print(sum)

CWE114_Process_Control 576
CWE121_Stack_Based_Buffer_Overflow 5064
CWE122_Heap_Based_Buffer_Overflow 6146
CWE123_Write_What_Where_Condition 288
CWE124_Buffer_Underwrite 2048
CWE126_Buffer_Overread 1452
CWE127_Buffer_Underread 2048
CWE134_Uncontrolled_Format_String 2880
CWE15_External_Control_of_System_or_Configuration_Setting 48
CWE176_Improper_Handling_of_Unicode_Encoding 48
CWE188_Reliance_on_Data_Memory_Layout 36
CWE190_Integer_Overflow 3960
CWE191_Integer_Underflow 2952
CWE194_Unexpected_Sign_Extension 1152
CWE195_Signed_to_Unsigned_Conversion_Error 1152
CWE196_Unsigned_to_Signed_Conversion_Error 18
CWE197_Numeric_Truncation_Error 894
CWE222_Truncation_of_Security_Relevant_Information 18
CWE223_Omission_of_Security_Relevant_Information 18
CWE226_Sensitive_Information_Uncleared_Before_Release 72
CWE23_Relative_Path_Traversal 2400
CWE242_Use_of_Inherently_Dangerous_Function 18
CWE244_Heap_Inspection 72
CWE247_Reliance_on_DNS_Lookups_in_Security_Decision 18
CWE252_Unchecked_Return_Val

In [81]:
# quét tất cả các file có trong source (Chỉ lấy những file CWE) / coi hiện tại đang coi như những file header k tồn tại
all_file=list(glob.glob("./dataset/*/CWE*.c")) + list(glob.glob("./dataset/*/*/CWE*.c"))+list(glob.glob("./dataset/*/*/*/CWE*.c")) 
all_file += list(glob.glob("./dataset/*/CWE*.cpp")) + list(glob.glob("./dataset/*/*/CWE*.cpp"))+list(glob.glob("./dataset/*/*/*/CWE*.cpp"))
all_file=list(set(all_file))
len(all_file) #Sô file quét được

100883

In [79]:
# dựa trên dataset mapping với file
all_file={os.path.basename(i):i for i in all_file }
data_final['path']=data_final['File'].map(all_file)

In [80]:
# xác định những file k được quét theo all_file
# sẽ loại những file header, những file main
#! cần clear theo line 
data_final[data_final.path.isna()].shape

212744

In [ ]:
# clear những file k tìm được path
data_final=data_final[~data_final.path.isna()]

In [ ]:
# đếm kết quả theo từng CWE
sum = 0
for cwe in data_final['CWE'].unique():
  print(cwe, len(data_final[data_final['CWE'] == cwe]))
  sum += len(data_final[data_final['CWE'] == cwe])
print(sum)

In [1]:
import json
import pandas  as pd 
import os,sys,glob 


dataset_phase2 = pd.read_csv('../example_datasets/test.csv')

In [2]:
dataset_phase2['CWE'] = dataset_phase2.File.apply(lambda x:x.split("__")[0])

In [3]:
dataset_phase2.line_error

0                [121]
1                [124]
2                [124]
3                [131]
4                [131]
             ...      
52472    [10000000000]
52473    [10000000000]
52474            [154]
52475    [10000000000]
52476    [10000000000]
Name: line_error, Length: 52477, dtype: object

In [4]:
# Đây chỉ tính là trên tất cả các file
# Đẩy sẽ
cwe_exist_line_error = dataset_phase2[dataset_phase2.line_error != '[10000000000]']
cwe_exist_line_error

,File,source_code,line_error,error_type,CWE
0,CWE114_Process_Control__w32_wchar_t_connect_so...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[121],[1],CWE114_Process_Control
1,CWE114_Process_Control__w32_wchar_t_connect_so...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[124],[1],CWE114_Process_Control
2,CWE114_Process_Control__w32_wchar_t_connect_so...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[124],[1],CWE114_Process_Control
3,CWE114_Process_Control__w32_wchar_t_connect_so...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[131],[1],CWE114_Process_Control
4,CWE114_Process_Control__w32_wchar_t_connect_so...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[131],[1],CWE114_Process_Control
...,...,...,...,...,...
52460,CWE90_LDAP_Injection__w32_wchar_t_listen_socke...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[56],[119],CWE90_LDAP_Injection
52462,CWE90_LDAP_Injection__w32_wchar_t_listen_socke...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[51],[119],CWE90_LDAP_Injection
52466,CWE90_LDAP_Injection__w32_wchar_t_listen_socke...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[51],[119],CWE90_LDAP_Injection
52470,CWE90_LDAP_Injection__w32_wchar_t_listen_socke...,/* TEMPLATE GENERATED TESTCASE FILE\nFilename:...,[154],[119],CWE90_LDAP_Injection


In [5]:
# Các file được định nghĩa line với minitest
sum = 0
for cwe in cwe_exist_line_error['CWE'].unique():
  #print(cwe, len(cwe_exist_line_error[cwe_exist_line_error['CWE'] == cwe]))
  if cwe.split("_")[0] == 'CWE401':
    print(cwe, len(cwe_exist_line_error[cwe_exist_line_error['CWE'] == cwe]))
  sum += len(cwe_exist_line_error[cwe_exist_line_error['CWE'] == cwe])
print(sum)

CWE401_Memory_Leak 829
31948


In [6]:
cwe_no_exist_line_error = dataset_phase2[dataset_phase2.line_error == '[10000000000]'].to_csv('no_exist_line_error.csv')
cwe_no_exist_line_error